In [1]:
import math
import numpy as np
import torch
import torch.nn.functional as F
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

### Задание 1

Ниже даны матрица $X$ (каждая строка - эмбеддинг очередного токена) и матрицы проекций $W_Q$, $W_K$ и $W_V$.

In [2]:
X = torch.tensor([[-1.51, -1.57,  0.87,  1.1 , -0.74],
                  [ 0.46, -0.4 , -0.64,  0.12, -0.02],
                  [-0.75,  0.44,  1.05,  0.38, -0.16]])
W_Q = torch.tensor([[ 1.88, -0.04, -1.51,  0.5 ,  0.18],
                    [-0.79,  1.06,  0.09, -0.3 ,  0.55],
                    [ 0.03,  0.44,  3.06, -1.91,  1.52],
                    [-0.1 , -2.17,  0.93,  0.82, -0.35],
                    [ 0.27,  0.54, -0.42, -0.8 ,  1.41]])
W_K = torch.tensor([[-0.03,  1.33, -1.91, -1.73,  0.73],
                    [ 1.06,  0.08,  1.01,  0.9 , -0.  ],
                    [ 0.17, -0.11, -0.11, -0.49,  0.7 ],
                    [-0.66, -1.44, -0.56,  0.95, -0.72],
                    [-0.5 , -1.2 ,  1.59, -0.47, -0.34]])
W_V = torch.tensor([[-1.55, -1.48,  2.23,  0.57, -1.53],
                    [-1.45, -0.91, -1.69,  0.43,  0.44],
                    [-1.05,  0.19, -0.65, -0.34,  0.12],
                    [-1.29,  1.48,  0.18,  0.24,  0.83],
                    [ 2.12,  1.09,  0.79, -0.21, -0.95]])

**Вопрос:** Каким будет новое представление входной последовательности после применения self-attention?

Полезные операции со ссылками на документацию: [`torch.matmul`](https://pytorch.org/docs/stable/generated/torch.matmul.html), [`F.softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html), [`math.sqrt`](https://docs.python.org/3/library/math.html) и метод [`transpose`](https://pytorch.org/docs/stable/generated/torch.Tensor.transpose.html) у тензора.

In [3]:
Q = torch.matmul(X, W_Q)
K = torch.matmul(X, W_K)
V = torch.matmul(X, W_V)
a = torch.matmul(Q, K.transpose(dim0=1, dim1=0)) / math.sqrt(X.shape[1])
alpha = F.softmax(a, dim=1)
z = torch.matmul(alpha, V)
print(z)

tensor([[-0.3952,  2.8954, -2.4458, -0.9194,  2.6043],
        [ 0.5535,  4.1379, -1.5410, -1.2466,  3.0007],
        [-1.2202,  1.3002, -2.7521, -0.4466,  1.7936]])


### Задание 2

В этом задании и далее мы будем работать с библиотекой [`transformers`](https://huggingface.co/docs/transformers/index) от [`HuggingFace`](https://huggingface.co/). На сегодняшний день это, пожалуй, самая популярная и удобная библиотека для работы с моделями на базе трансформерной архитектуры.

В качестве [`модели`](https://huggingface.co/utrobinmv/t5_translate_en_ru_zh_small_1024) возьмем [`T5`](https://huggingface.co/docs/transformers/model_doc/t5), обученную на задачу перевода с одного языка на другой.

Прежде всего, загрузим токенизатор, который будет превращать текст в последовательность номеров токенов из словаря.

In [4]:
!pip install sentencepiece

In [5]:
tokenizer = AutoTokenizer.from_pretrained('utrobinmv/t5_translate_en_ru_zh_small_1024', use_fast=False)

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Для примера попробуем превратить строку "Мама мыла раму" в последовательность номеров токенов.
Для этого нужно вызвать [`tokenizer(text)`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__), где text - текст, который мы хотим токенизировать.

In [6]:
tokens = tokenizer('Мама мыла раму')
print(tokens)

{'input_ids': [43690, 290, 391, 4, 6587, 57, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


Видно, что токенизатор превратил текст в последовательность номеров токенов из словаря (поле input_ids), а также расчитал маску внимния (поле attention_mask).

Последовательность номеров токенов можно декодировать обратно в текст:

In [7]:
print(tokenizer.decode(tokens['input_ids']))

Мама мыла раму</s>


Видно, что токенизатор автоматически добавил `</s>` - токен конца входной последовательности. Попробуем декодировать каждый номер (token-id) отдельно:

In [8]:
for token_id in tokens['input_ids']:
    print(token_id, tokenizer.decode(token_id))

43690 Мама
290 мы
391 ла
4 
6587 рам
57 у
1 </s>


Видно, что мы имеем дело с [`subword-токенизацией`](https://huggingface.co/docs/transformers/tokenizer_summary#subword-tokenization) - некоторые слова представляются одним токеном, но некоторые разбиваются на несколько.

Можно заглянуть в словарь и напрямую:

In [9]:
token_to_id = tokenizer.get_vocab()
id_to_token = {token_id: token for token, token_id in token_to_id.items()}

for i in range(100):
    print(i, id_to_token[i])

0 <pad>
1 </s>
2 <unk>
3 ,
4 ▁
5 .
6 0
7 ▁the
8 1
9 -
10 2
11 ▁of
12 ▁and
13 s
14 ▁в
15 9
16 ▁и
17 3
18 5
19 ▁to
20 4
21 ▁in
22 6
23 /
24 8
25 ▁(
26 7
27 ▁a
28 )
29 的
30 。
31 :
32 а
33 ▁на
34 ▁с
35 ▁-
36 ▁is
37 ▁for
38 '
39 ▁"
40 和
41 ▁по
42 ▁that
43 、
44 е
45 ▁on
46 ▁with
47 (
48 и
49 ▁The
50 ;
51 ▁не
52 ▁что
53 ▁as
54 ▁by
55 ы
56 м
57 у
58 ▁was
59 在
60 ▁from
61 ing
62 ▁для
63 ▁be
64 ▁are
65 ▁В
66 "
67 ▁or
68 ▁at
69 о
70 ом
71 ed
72 ▁из
73 ▁it
74 ),
75 ▁к
76 年
77 ▁от
78 ▁о
79 ▁за
80 ▁«
81 ▁как
82 ов
83 ▁an
84 ).
85 ▁I
86 A
87 ▁you
88 d
89 ▁which
90 ▁not
91 ▁have
92 ▁или
93 t
94 х
95 ▁A
96 ▁this
97 ▁In
98 a
99 ?


Видно, что часть токенов начинается с нижнего подчеркивания. Это означает, что данный токен является началом какого-то слова и не может быть другой частью слова (например, стоять в середине или в конце). Посмотрим, например, как токенизируются строки "I and you" и "Iandyou":

In [10]:
print(tokenizer("I and you"))
print(tokenizer("Iandyou"))

{'input_ids': [85, 12, 87, 1], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [85, 370, 4760, 1], 'attention_mask': [1, 1, 1, 1]}


In [11]:
print(id_to_token[12])
print(id_to_token[370])

▁and
and


Видно, что токен "and" имеет две записи в словере (для начала слова и для других расположений). Но при этом при декодировании токенизатор убирает нижнее подчеркивание:

In [12]:
print(tokenizer.decode(12))
print(tokenizer.decode(370))

and
and


При токенизации можно попросить токенизатор не добавлять токен конца последовательности. Для этого нужно выставить `add_special_tokens=False`:

In [13]:
tokenizer('Мама мыла раму', add_special_tokens=False)

{'input_ids': [43690, 290, 391, 4, 6587, 57], 'attention_mask': [1, 1, 1, 1, 1, 1]}

**Вопрос:** Какой номер в словаре имеет токен "▁наука"?

In [35]:
vocab = tokenizer.get_vocab()
print(vocab['▁наука'])

24880


### Задание 3

Токенизируйте строку "LLM - это смысл моей жизни" с `add_special_tokens=False`. В какую последовательность номеров токенизатор перевел строку? 

In [36]:
print(tokenizer('LLM - это смысл моей жизни', add_special_tokens=False))

{'input_ids': [503, 16322, 35, 140, 12440, 8949, 737], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


### Задание 4

Теперь загрузим модель и сгенерируем перевод для предложения.

In [37]:
model = T5ForConditionalGeneration.from_pretrained('utrobinmv/t5_translate_en_ru_zh_small_1024')
print(model)

config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(65100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(65100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

Видно, что модель состоит из кодировщика и декодировщика. Посмотрим, сколько параметров она имеет:

In [38]:
print(model.num_parameters())

110724480


Переведем строку "Без труда не выловишь и рыбку из пруда." с русского языка на английский. Для этого нужно особым образом сконструировать запрос ([`примеры`](https://huggingface.co/utrobinmv/t5_translate_en_ru_zh_small_1024)):

In [40]:
src_text = 'translate to en: Без труда не выловишь и рыбку из пруда.'

Превратим текст в номера токенов:

In [41]:
x = tokenizer(src_text, return_tensors='pt')
print(x)

{'input_ids': tensor([[21809,    19,  1904,    31, 10401,  3306,    51,   160,  2406,  9466,
            16,  6596,   435,    72, 46769,    32,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Генерируем выходную последовательность:

In [42]:
output_ids = model.generate(x['input_ids'])
print(output_ids)

tensor([[    0, 20040,   327,     3,  1182,    38,    93, 15373,     7,  3958,
           217,    11,     7,     4, 18263,     5,     1]])


Превращаем последовательность номеров токенов в текст:

In [43]:
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

Without work, don't catch the fish out of the pond.


**Вопрос:** Какой перевод модель сгенерирует для текста "Однажды, в студеную зимнюю пору. Я из лесу вышел; был сильный мороз. Гляжу, поднимается медленно в гору. Лошадка, везущая хворосту воз."?

In [44]:
src_text = 'translate to en: Однажды, в студеную зимнюю пору. Я из лесу вышел; был сильный мороз. Гляжу, поднимается медленно в гору. Лошадка, везущая хворосту воз.'
x = tokenizer(src_text, return_tensors='pt')
output_ids = model.generate(x['input_ids'])
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

One day, I went out of the woods; there was a strong frost. I'm going to climb slowly into the mountain. A horse that is lucky for you.


### Задание 5

Попробуем обучить модель генерировать ответы на следующие вопросы:
1. Какой город является столицей России?
2. Какой город является столицей Соединенных Штатов Америки?

Как обычно, для пары (вопрос, ответ) получить loss и минимизировать его. Для примера получим loss для для пары ("Какой город является столицей Финляндии?", "Хельсинки").

In [45]:
src_text = "Какой город является столицей Финляндии?"
tgt_text = "Хельсинки"

Превратим входной и выходной тексты в последовательности номеров токенов.

In [46]:
src_ids = tokenizer(src_text, return_tensors='pt')
tgt_ids = tokenizer(tgt_text, return_tensors='pt')

print(src_ids)
print(tgt_ids)

{'input_ids': tensor([[  743,   108,  2143,   277, 37909, 15859,    99,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[37136,  5749,   182,     1]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


Запустим модель, передав ей входную и выходную последовательность.

In [47]:
with torch.no_grad():
    output = model(input_ids=src_ids['input_ids'], labels=tgt_ids['input_ids'])
print(output.keys())

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])


Видим, что модель вернула loss для выходной последовательности, а также логиты для данного токена (чтобы мы могли посчитать какой-нибудь свой loss, если захотим). Посмотрим, чему равен loss.

In [48]:
print(output.loss)

tensor(10.3770)


Лосс получился очень большой. Посмотрим, какой ответ модель сейчас генерирует на наш вопрос.

In [49]:
print(tokenizer.decode(model.generate(src_ids['input_ids'])[0], skip_special_tokens=True))

What kind of city is the capital of Finland?


Модель сгенерировала перевод вопроса, что неудивительно, так как ее учили только переводить тексты.

**Вопрос**: Какой loss у пары ("translate to en: Вчера мы ходили в зоопарк и видели там смешных капибар.", "Yesterday we went to the zoo and saw funny capybaras there.")?

In [50]:
src_text = "translate to en: Вчера мы ходили в зоопарк и видели там смешных капибар."
tgt_text = "Yesterday we went to the zoo and saw funny capybaras there."
    
src_ids = tokenizer(src_text, return_tensors='pt')
tgt_ids = tokenizer(tgt_text, return_tensors='pt')
with torch.no_grad():
    output = model(input_ids=src_ids['input_ids'], labels=tgt_ids['input_ids'])
print(output.loss)
print(...)

tensor(1.4681)
Ellipsis


Приступим, наконец, к обучению. Подготовим данные.

In [ ]:
input_texts = ["Какой город является столицей России?", "Какой город является столицей Новой Зеландии?"]
output_texts = ["Москва", "Веллингтон"]

train_data = []
for input_text, output_text in zip(input_texts, output_texts):
    x = tokenizer(input_text)
    y = tokenizer(output_text)
    x['labels'] = y['input_ids']
    train_data.append(x)
print(train_data)

Создадим data collator для выравнивания последовательностей по максимальной длине с использованием pad-токенов и их маскировки.

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

Посмотрим, что он делает.

In [ ]:
print(data_collator(train_data))

Видно, что
* Массив словарей превратился в "словарь массивов": в каждом поле содержится батч.
* input_ids дополнились до максимальной длины нулями (все последовательности теперь имеют одну и ту же длину).
* attention_mask тоже дополнился нулями (не будем учитывать pad-токены)
* labels дополнился -100 (не будем считать loss по pad-токенам)

Запускаем обучение.

In [ ]:
train_args = Seq2SeqTrainingArguments(
    './output',
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    max_steps=100,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

In [ ]:
model.train(); # Переводим модель в режим обучения (включаем Dropout, ...)

In [ ]:
trainer = Seq2SeqTrainer(model, train_args, train_dataset=train_data, data_collator=data_collator)

In [ ]:
trainer.train()

Проверяем, научилась ли наша модель отвечать на два вопроса.

In [ ]:
model.eval(); # Переводим модель в режим инференса (Отключаем Dropout, ...)

In [ ]:
src_ids = tokenizer('"Какой город является столицей России?"', return_tensors='pt')
print(tokenizer.decode(model.generate(src_ids['input_ids'])[0], skip_special_tokens=True))

src_ids = tokenizer('"Какой город является столицей Новой Зеландии?"', return_tensors='pt')
print(tokenizer.decode(model.generate(src_ids['input_ids'])[0], skip_special_tokens=True))